In [1]:
import pandas as pd
import mysql.connector
from scipy.stats import ttest_ind
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats


## Connecting to DataBase

In [2]:
db = mysql.connector.connect(
   host="",
  user="",
  password="",
  database="Litopia")
mycursor = db.cursor()

# 2 : Impact of discount on Sales

In [240]:
sql_query = """
SELECT
    o.Quantity,
    o.discount,
    o.Price
FROM product AS p
INNER JOIN orders AS o ON p.ID = o.Product_ID;
"""
mycursor.execute(sql_query)

In [241]:
columns = [i[0] for i in mycursor.description]
result = mycursor.fetchall()
laptop_df= pd.DataFrame(result, columns=columns)


In [242]:
discount_df = laptop_df[(laptop_df['discount'] > 0) ]
without_discount_df = laptop_df[(laptop_df['discount'] == 0)]
discount_sample = discount_df.sample(n=100000,random_state=4)
without_discount_sample = without_discount_df.sample(n=100000,random_state=4)

In [249]:
print(without_discount_sample['Quantity'].sum(),discount_sample['Quantity'].sum())

110694 110606


## Total Quantity :
- With Discouint : 110606
- Without Discount: 110694

In [245]:
print(discount_df['Quantity'].mean(),without_discount_df['Quantity'].mean())

1.105427429675869 1.1064411244853478


## Avg Quantity : 
- With Discount :1.105427429675869 
- Without Discount : 1.1064411244853478

In [246]:
quantity_discount= discount_sample ['Quantity'].tolist()
quantity_without_discount = without_discount_sample['Quantity'].tolist()
quantity_discount = [int(item) for item in quantity_discount ]
quantity_without_discount = [int(item) for item in quantity_without_discount ]

In [247]:
t_statistic, p_value = ttest_ind(quantity_without_discount,quantity_discount, equal_var=True)
print("T-Statistic:", t_statistic)
print("P-Value:", p_value)

T-Statistic: 0.5408065211877862
P-Value: 0.5886415498384536


## T-Statistic:
- The t-statistic measures the difference between the means of the two groups (quantity with discounts and quantity without discounts) relative to the variation within each group. 
- A t-statistic of 0.5408 suggests that the difference in quantity is not significantly large.

## P-Value:
- The p-value represents the probability that you would observe the data you have if there were no real difference between the groups. In this case, a p-value of 0.5886 is relatively high. It means that there is a 58.86% chance of observing these results (or more extreme results) if there were no significant difference in the quantities between the two groups.

- Since the p-value is greater than a typical significance level of 0.05 (a commonly chosen value), you would fail to reject the null hypothesis. The null hypothesis (H0) in this context would be that there is no significant difference in the quantities of items sold with and without discounts.

# 3 : Impact of discount on profit 

- I used the profit percentage relative to the original price for better accuracy :(Profit/Total_price)*100 

In [257]:
sql_query = """
SELECT p.Model_Name, (o.Profit/o.Total_price)*100 as Profit, o.discount, o.Total_price
FROM product AS p
INNER JOIN orders AS o ON p.ID = o.Product_ID
WHERE o.Total_price > o.Profit 
"""
mycursor.execute(sql_query)


In [258]:
columns = [i[0] for i in mycursor.description]
result = mycursor.fetchall()
df= pd.DataFrame(result, columns=columns)


In [259]:
df_discount = df[df['discount']>0]
df_without_discount=df[df['discount']==0]
discount_sample = df_discount.sample(n=100000,random_state=4)
without_sample = df_without_discount.sample(n=100000,random_state=4)

In [260]:
profit_discount= discount_sample ['Profit'].tolist()
profit_without_discount =without_sample ['Profit'].tolist()
profit_discount = [float(item) for item in profit_discount ]
profit_without_discount = [float(item) for item in profit_without_discount ]

In [261]:
t_statistic, p_value = ttest_ind(profit_without_discount,profit_discount, equal_var=True)
print("T-Statistic:", t_statistic)
print("P-Value:", p_value)

T-Statistic: 1.8195569166129457
P-Value: 0.06882800207980423


 ## T-Statistic:
- The T-Statistic measures the difference between the means of the two groups (profit with discounts and profit without discounts).
- In your case, a T-Statistic of 0.3530577962510027 a negative T-statistic indicates that the sample mean of the first group (likely the profits without discounts) is lower than the sample mean of the second group (likely the profits with discounts).
## P-Value:
-  The P-Value is a measure of the probability of observing such extreme results (or more extreme) if there were no significant difference between the two groups.
- With a P-value of P-Value: 0.7240454838131373(which is greater than the typical significance level of 0.05), in this case, with a p-value of 0.7240454838131373, it is quite high, suggesting that the observed effect is not statistically significant at a common significance level of 0.05. 

# 4 : Budget allocation among brands & categories

## Category : 

In [3]:
sql_query = """
select o.Profit,c.Category_Name as Category
from Product as p
inner join Orders as o on p.id=o.product_id
inner join Category as c on p.category_id=c.id;
"""
mycursor.execute(sql_query)

In [4]:
columns = [i[0] for i in mycursor.description]
result = mycursor.fetchall()
category_df= pd.DataFrame(result, columns=columns)


In [20]:
convertible_df = category_df[category_df['Category']=='2 in 1 Convertible']
notebook_df = category_df[category_df['Category']=='Notebook']
gaming_df = category_df[category_df['Category']=='Gaming']
netbook_df= category_df[category_df['Category']=='Netbook']
workstation_df = category_df[category_df['Category']=='Workstation']
ultrabook_df = category_df[category_df['Category']=='Ultrabook']


In [21]:
convertible = convertible_df['Profit'].tolist()
notebook = notebook_df['Profit'].tolist()
gaming = gaming_df['Profit'].tolist()
netbook = netbook_df['Profit'].tolist()
workstation = workstation_df['Profit'].tolist()
ultrabook = ultrabook_df['Profit'].tolist()

In [22]:
convertible =[float(item) for item in  convertible]
notebook=[float(item) for item in notebook]
gaming =[float(item) for item in  gaming]
netbook=[float(item) for item in  netbook]
workstaion=[float(item) for item in  workstation]
ultrabook =[float(item) for item in ultrabook]

In [29]:
Categories = [ultrabook,gaming,netbook,workstation,notebook]
for category in range (0,len(Categories)) :
    t_statistic, p_value = ttest_ind(convertible,Categories[category],equal_var=False)
    if p_value < 0.05 :
        print ("P-Value:", p_value,'\n',"T-Statistic:", t_statistic,'\n','There is a significant difference between these two category%s'%category,'\n')
    else :
        print ('There isnt significant difference between these two category %s '%category)
    
    

P-Value: 1.5182426410695703e-188 
 T-Statistic: -29.316021539590917 
 There is a significant difference between these two category0 

P-Value: 5.083926076186383e-154 
 T-Statistic: -26.458594834421167 
 There is a significant difference between these two category1 

P-Value: 3.588023195610943e-17 
 T-Statistic: 8.442687459010784 
 There is a significant difference between these two category2 

P-Value: 1.976022155275232e-198 
 T-Statistic: -30.83661332747331 
 There is a significant difference between these two category3 

P-Value: 0.0 
 T-Statistic: 46.023032340990824 
 There is a significant difference between these two category4 



## T-Statistic &  P-Value :
- Based on the t-statistics and p-value and compare them to eachother there is a significant difference between different laptop category 
- So based on compare them by profit : Workstation > Ultrabook > Gaming > Convertible > netbook > Notbook 

## Brand : 

In [30]:
sql_query = """
select o.Profit ,m.Manufacturer_Name as Brand
from Product as p
inner join Orders as o on p.id=o.product_id
inner join Manufacturer as m on p.manufacturer_id = m.id;
"""
mycursor.execute(sql_query)

In [31]:
columns = [i[0] for i in mycursor.description]
result = mycursor.fetchall()
brand_df= pd.DataFrame(result, columns=columns)


In [32]:
lg_df = brand_df[brand_df['Brand'] == 'LG']
razer_df =brand_df[brand_df['Brand'] == 'Razer']
google_df = brand_df[brand_df['Brand'] == 'Google']
samsung_df = brand_df[brand_df['Brand'] == 'Samsung']
huawei_df = brand_df[brand_df['Brand'] == 'Huawei']
msi_df = brand_df[brand_df['Brand'] == 'MSI']
microsoft_df = brand_df[brand_df['Brand'] == 'Microsoft']
apple_df =brand_df[brand_df['Brand'] == 'Apple']
toshiba_df = brand_df[brand_df['Brand'] == 'Toshiba']
xiaomi_df = brand_df[brand_df['Brand'] == 'Xiaomi']
asus_df = brand_df[brand_df['Brand'] == 'Asus']
hp_df = brand_df[brand_df['Brand'] == 'HP']
lenovo_df = brand_df[brand_df['Brand'] == 'Lenovo']
acer_df = brand_df[brand_df['Brand'] == 'Acer']
fujitsu_df = brand_df[brand_df['Brand'] == 'Fujitsu']
chuwi_df =brand_df[brand_df['Brand'] == 'Chuwi']
vero_df=brand_df[brand_df['Brand'] == 'Vero']
mediacom_df=brand_df[brand_df['Brand'] == 'Mediacom']
dell_df=brand_df[brand_df['Brand'] == 'Dell']


In [34]:
lg = lg_df['Profit'].tolist()
razer =razer_df['Profit'].tolist()
google= google_df['Profit'].tolist()
samsung = samsung_df['Profit'].tolist()
huawei= huawei_df['Profit'].tolist()
msi= msi_df['Profit'].tolist()
microsoft =microsoft_df['Profit'].tolist()
apple =apple_df['Profit'].tolist()
toshiba= toshiba_df['Profit'].tolist()
xiaomi= xiaomi_df['Profit'].tolist()
asus = asus_df['Profit'].tolist()
hp= hp_df['Profit'].tolist()
lenovo= lenovo_df['Profit'].tolist()
acer= acer_df['Profit'].tolist()
fujitsu= fujitsu_df['Profit'].tolist()
chuwi=chuwi_df['Profit'].tolist()
vero=vero_df['Profit'].tolist()
mediacom=mediacom_df['Profit'].tolist()
dell=dell_df['Profit'].tolist()


In [35]:
lg = [float(item) for item in lg]
razer =[float(item) for item in  razer]
google=[float(item) for item in  google]
samsung =[float(item) for item in  samsung]
huawei= [float(item) for item in  huawei]
msi=[float(item) for item in  msi]
microsoft =[float(item) for item in microsoft]
apple =[float(item) for item in  apple]
toshiba= [float(item) for item in  toshiba]
xiaomi= [float(item) for item in xiaomi]
asus =[float(item) for item in asus]
hp= [float(item) for item in  hp]
lenovo= [float(item) for item in  lenovo]
acer= [float(item) for item in acer]
fujitsu= [float(item) for item in fujitsu]
chuwi=[float(item) for item in  chuwi]
vero=[float(item) for item in vero]
mediacom=[float(item) for item in mediacom]
dell=[float(item) for item in dell]

In [39]:
Brands_Name = ['Asus', 'Lenovo', 'Samsung', 'HP', 'Acer', 'Fujitsu', 'Dell',
       'Google', 'Razer', 'Xiaomi', 'Toshiba', 'Chuwi', 'MSI',
       'Microsoft', 'Apple', 'Vero', 'Huawei', 'Mediacom']
Brands=[asus,lenovo,samsung,hp,acer,fujitsu,dell,google,razer,xiaomi,toshiba,chuwi,msi,microsoft,apple,vero,huawei,mediacom]
for brand in range (0,len(Brands)) :
    t_statistic, p_value = ttest_ind(lg,Brands[brand],equal_var=False)
    if p_value < 0.05 :
        print ("P-Value:", p_value,'\n',"T-Statistic:", t_statistic,'\n','There is a significant difference LG  and %s'%Brands_Name[brand],'\n')
    else :
        print ("P-Value:", p_value,'\n',"T-Statistic:", t_statistic,'\n','There isnt significant difference LG  and %s'%Brands_Name[brand],'\n')
    

P-Value: 3.56119970170991e-26 
 T-Statistic: 11.007802420281225 
 There is a significant difference LG  and Asus 

P-Value: 3.0915575759148107e-29 
 T-Statistic: 11.73357290594417 
 There is a significant difference LG  and Lenovo 

P-Value: 4.3199101407972377e-10 
 T-Statistic: 6.315061004712222 
 There is a significant difference LG  and Samsung 

P-Value: 1.0994639706621605e-28 
 T-Statistic: 11.60541664917879 
 There is a significant difference LG  and HP 

P-Value: 1.2401301256650978e-41 
 T-Statistic: 14.417563091786807 
 There is a significant difference LG  and Acer 

P-Value: 3.9530431209389615e-42 
 T-Statistic: 14.472816175920398 
 There is a significant difference LG  and Fujitsu 

P-Value: 9.135517958886744e-29 
 T-Statistic: 11.624338588246657 
 There is a significant difference LG  and Dell 

P-Value: 2.974193099351573e-07 
 T-Statistic: 5.159253575326069 
 There is a significant difference LG  and Google 

P-Value: 0.10919736627745567 
 T-Statistic: 1.6027387407248535 


## T-Statistic &  P-Value :
- So based on the ttest the top most profit is for LG and Razer ( no difference )


# 5 : Priority Challenge : Worth it or not ?!

In [ ]:
orders_priority = []

for priority in range(1, 5):
    sql_query = f"""
    SELECT Ship_Duration
    FROM orders
    WHERE Priority_ID = {priority};
    """

    mycursor.execute(sql_query)
    results = mycursor.fetchall()

    priority_column = [result[0] for result in results]
    orders_priority.append(priority_column)

In [ ]:
df = pd.DataFrame(orders_priority).transpose()
df.columns = ['c', 'l', 'h', 'm']
df = df[['c', 'h', 'm', 'l']]
df.describe()

In [ ]:
def reject_null_hypothesis(p_value, alpha):
    if p_value <= alpha:
        return True
    else:
        return False

In [ ]:
test_result = stats.kruskal(orders_priority[0], orders_priority[1], orders_priority[2], orders_priority[3])
print(test_result)
reject_null_hypothesis(test_result[1], 0.01)